In [5]:
import csv
import datetime
import numpy as np
import pandas as pd
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep
from bs4 import BeautifulSoup
from requests import get

In [2]:
df = pd.read_excel('data/base_data.xlsx')
playersSet = set(df['Winner']).union(set(df['Loser']))
players = {n : [] for n in playersSet}

In [3]:
def write_csv(players):
    with open('data/playersdata.csv', 'w', newline='', encoding='utf-8-sig') as file :
        writer = csv.writer(file)
        writer.writerow(['Rank', 'Player', 'Elo', 'HardRaw','ClayRaw', 'GrassRaw', 
                         'hElo', 'cElo', 'gElo', 'BirthDate', 'Hand', 'BackHand'])
        for p in players.values():
            writer.writerow(p)

In [4]:
def uniformName(w):
    surname = w.split()
    newName = surname[-1] + ' '
    for n in range(len(surname)-2, -1, -1):
        newName += surname[n][0] + '.'
    return newName

In [5]:
def static_scraping(players):
    url = 'http://tennisabstract.com/reports/atp_elo_ratings.html'
    r =  get(url)
    page = BeautifulSoup(r.content)
    table = page.find(id='reportable')
    allTr = table.find_all('tr')

    for tr in allTr[1:] :
        tds = tr.find_all('td')
        toWrite = [tds[i].text for i in (0, 1, 3, 5, 6, 7, 9, 10, 11)]

        toWrite[1] = uniformName(toWrite[1])

        playerLink = tr.a['href']
        raw = get(playerLink)
        playerPage = BeautifulSoup(raw.content)
        s = playerPage.find('script', language='JavaScript').text
        i = s.find('var dob =')
        dob = datetime.date(int(s[i+10:i+14]), int(s[i+14:i+16]), int(s[i+16:i+18]))
        i = s.find('var hand =')
        hand = s[i+12]
        if hand not in {'L', 'R'}:
            hand = None
        i = s.find('var backhand =')
        backhand = s[i+16]
        if backhand not in {'1', '2'}:
            backhand = None

        toWrite += [dob, hand, backhand]
        players[toWrite[1]] = toWrite

In [6]:
def dynamic_scraping(playerNames):
    driver = webdriver.Chrome()
    driver.get("https://www.atptour.com/en/players")
    items = list(playerNames.items())
    for w, values in items:
        surname = w.split()[0]
        search = driver.find_element_by_name("playerInput")
        search.clear()
        search.click()
        search.send_keys(surname+ " ")
        ul = driver.find_element_by_id("playerDropdown")
        sleep(8)
        
        try:   
            playersList = ul.find_elements_by_partial_link_text(" " + surname)
            for p in playersList:
                pName = p.text.replace('-', ' ').split()
                initials = w.split()[1].split('.')[:-1]
                print(pName , ' --> ' , initials)
                found = True
                for i in range(len(initials)):
                    found = found and initials[i] == pName[i][0]
                if found:
                    player = p
                    break
                    
            if not found:    
                raise Exception(w + ' not found')
                
            print(w + ": " + player.text)
            player.click()
            sleep(8)
                 
            playerInfo = driver.find_element_by_css_selector("div.player-profile-hero-table")
            pBD = playerInfo.find_element_by_css_selector("span.table-birthday").text
            pBD = pBD[1:-1].replace('.', '-')

            play = driver.find_element_by_xpath('//*[@id="playerProfileHero"]/div[2]/div[2]/div/table/tbody/tr[2]/td[3]/div/div[2]').text
            hand = play[0]
            if hand not in {'L', 'R'}:
                        hand = None
            index = play.find('-Handed Backhand')
            backHand = play[index-3 : index]
            if backHand == "One" : 
                backHand = 1
            elif backHand == "Two":
                backHand = 2
            else:
                backHand = None
        except Exception as e:
            print(e)
            pBD = None
            hand = None
            backHand = None
            
        if values == []:
            playerNames[w] = [None, w, None, None, None, None, None, None, None, pBD, hand, backHand]
        else:
            values[-3] = pBD
            values[-2] = hand
            values[-1] = backHand
            playerNames[w] = values

In [7]:
print("Static Scraping started")
static_scraping(players)
print("Dynamic Scraping started")
dynamic_scraping(players)
print("Writing...")
write_csv(players)
print("Done")

Static Scraping started
Dynamic Scraping started
['Alex', 'Mayer']  -->  ['F']
['Clemens', 'Mayer']  -->  ['F']
['Florian', 'Mayer']  -->  ['F']
Mayer F.: Florian Mayer
['Norbert', 'Gombos']  -->  ['N']
Gombos N.: Norbert Gombos
['Evgeny', 'Kirillov']  -->  ['E']
Kirillov E.: Evgeny Kirillov
['Guillermo', 'Olaso']  -->  ['G']
Olaso G.: Guillermo Olaso
['Yassine', 'Idmbarek']  -->  ['Y']
Idmbarek Y.: Yassine Idmbarek
string index out of range
['Alexander', 'Sadecky']  -->  ['A']
Sadecky A.: Alexander Sadecky
['Gonzalo', 'Lama']  -->  ['G']
Lama G.: Gonzalo Lama
['Chia', 'Cheng', 'Yang']  -->  ['T', 'H']
['Shao', 'Chi', 'Yang']  -->  ['T', 'H']
['Dae', 'Yang', 'Kweon']  -->  ['T', 'H']
['Zi', 'Dong', 'Yang']  -->  ['T', 'H']
['Eui', 'Yeol', 'Yang']  -->  ['T', 'H']
['Yi', 'Fan', 'Yang']  -->  ['T', 'H']
['Wei', 'Guang', 'Yang']  -->  ['T', 'H']
['Hao', 'Yang', 'Chen']  -->  ['T', 'H']
['Tsung', 'Hua', 'Yang']  -->  ['T', 'H']
Yang T.H.: Tsung-Hua Yang
['Dmitry', 'Tursunov']  -->  ['D']
T

['Filip', 'Horansky']  -->  ['F']
Horansky F.: Filip Horansky
['Adrian', 'Garza']  -->  ['D']
['Daniel', 'Garza']  -->  ['D']
Garza D.: Daniel Garza
['Gerard', 'Granollers']  -->  []
Granollers Pujol G.: Gerard Granollers
['Chris', 'Herbert']  -->  ['P', 'H']
['Pierre', 'Hugues', 'Herbert']  -->  ['P', 'H']
Herbert P.H.: Pierre-Hugues Herbert
['Adam', 'Becker']  -->  ['B']
['Benjamin', 'Becker']  -->  ['B']
Becker B.: Benjamin Becker
Message: stale element reference: element is not attached to the page document
  (Session info: chrome=79.0.3945.88)

['Alex', 'Kuznetsov']  -->  ['A']
Kuznetsov A.: Alex Kuznetsov
['Marco', 'Chiudinelli']  -->  ['M']
Chiudinelli M.: Marco Chiudinelli
['Agustin', 'Coria']  -->  ['F']
['Federico', 'Coria']  -->  ['F']
Coria F.: Federico Coria
['Germain', 'Gigounon']  -->  ['G']
Gigounon G.: Germain Gigounon
['Jason', 'Kubler']  -->  ['J']
Kubler J.: Jason Kubler
['Andrew', 'Harrison']  -->  ['R']
['Chris', 'Harrison']  -->  ['R']
['Christian', 'Harrison']  

['Felix', 'Auger', 'Aliassime']  -->  ['F']
Auger-Aliassime F.: Felix Auger-Aliassime
['Joao', 'Domingues']  -->  ['J']
Domingues J.: Joao Domingues
['Apostolos', 'Tsitsipas']  -->  ['S']
['Petros', 'Tsitsipas']  -->  ['S']
['Stefanos', 'Tsitsipas']  -->  ['S']
Tsitsipas S.: Stefanos Tsitsipas
['Adam', 'Novak']  -->  ['D']
['Balazs', 'Novak']  -->  ['D']
['Bosko', 'Novakovic']  -->  ['D']
['David', 'Novak']  -->  ['D']
Novak D.: David Novak
string index out of range
['Jeevan', 'Nedunchezhiyan']  -->  ['J']
Nedunchezhiyan J.: Jeevan Nedunchezhiyan
['Mikhail', 'Elgin']  -->  ['M']
Elgin M.: Mikhail Elgin
['Bowen', 'Ouyang']  -->  ['B']
Ouyang B.: Bowen Ouyang
['Mehdi', 'Ziadi']  -->  ['M']
Ziadi M.: Mehdi Ziadi
string index out of range
['Marco', 'Viola']  -->  ['M']
Viola M.: Marco Viola
string index out of range
['Adam', 'Anderson']  -->  ['K']
['Alec', 'Anderson']  -->  ['K']
['Andrew', 'Anderson']  -->  ['K']
['Andrew', 'Anderson']  -->  ['K']
['Art', 'Anderson']  -->  ['K']
['Derek'

['David', 'Simon']  -->  ['G']
['Ezequiel', 'Simonit']  -->  ['G']
['Francotenis', 'De', 'Simone']  -->  ['G']
['Frantisek', 'Simon']  -->  ['G']
['Fred', 'Simonsson']  -->  ['G']
['Frederik', 'Simon']  -->  ['G']
['George', 'Simond']  -->  ['G']
Simon G.: George Simond
string index out of range
['Wishaya', 'Trongcharoenchaikul']  -->  ['W']
Trongcharoenchaikul W.: Wishaya Trongcharoenchaikul
['Adrian', 'Clarke']  -->  ['J']
['Adrian', 'Clarke']  -->  ['J']
['B.L.', 'Clarke']  -->  ['J']
['Braden', 'Clarke']  -->  ['J']
['Chris', 'Clarke']  -->  ['J']
['Curtis', 'Clarke']  -->  ['J']
['Donaldson', 'Clarke']  -->  ['J']
['E.R.', 'Clarke']  -->  ['J']
['Edwin', 'Clarke']  -->  ['J']
['G.', 'Clarke']  -->  ['J']
['Jaime', 'Clarke']  -->  ['J']
Clarke J.: Jaime Clarke
string index out of range
['Bill', 'Quigley']  -->  ['E']
['Eric', 'Quigley']  -->  ['E']
Quigley E.: Eric Quigley
['Andreas', 'Seppi']  -->  ['A']
Seppi A.: Andreas Seppi
['A', 'El', 'Ghani']  -->  []
El Amrani R.: A El Ghan

['Laslo', 'Djere']  -->  ['L']
Djere L.: Laslo Djere
['Ai', 'Min', 'Zhang']  -->  ['Z']
['Andrew', 'Zhang']  -->  ['Z']
['Bo', 'Zhang']  -->  ['Z']
['Bolun', 'Zhang']  -->  ['Z']
['Changli', 'Zhang']  -->  ['Z']
['Changli', 'Zhang']  -->  ['Z']
['Dongrun', 'Zhang']  -->  ['Z']
['Nian', 'Fei', 'Zhang']  -->  ['Z']
['Hao', 'Zhang']  -->  ['Z']
['Hao', 'Zhang']  -->  ['Z']
['Jia', 'Heng', 'Zhang']  -->  ['Z']
['Yun', 'Hong', 'Zhang']  -->  ['Z']
['Jiu', 'Hua', 'Zhang']  -->  ['Z']
['Ting', 'Hui', 'Zhang']  -->  ['Z']
['Hui', 'Zhang']  -->  ['Z']
['Jiajun', 'Zhang']  -->  ['Z']
['Zheng', 'Qi', 'Jonathan', 'Zhang']  -->  ['Z']
Zhang Z.: Zheng Qi Jonathan Zhang
string index out of range
['Federico', 'Gaio']  -->  ['F']
Gaio F.: Federico Gaio
['Michael', 'Berrer']  -->  ['M']
Berrer M.: Michael Berrer
['Illya', 'Marchenko']  -->  ['I']
Marchenko I.: Illya Marchenko
['Daniel', 'Dutra', 'da', 'Silva']  -->  []
Dutra Silva R.: Daniel Dutra da Silva
['Gerald', 'Melzer']  -->  ['G']
Melzer G.: Ger

...

['A.', 'Baker']  -->  ['J']
['A.T.', 'Baker']  -->  ['J']
['A.T.', 'Baker2']  -->  ['J']
['A.T.', 'Baker2']  -->  ['J']
['Adam', 'Baker']  -->  ['J']
['Blu', 'Baker']  -->  ['J']
['Brian', 'Baker']  -->  ['J']
['Jean', 'Edouard', 'Baker']  -->  ['J']
Baker J.: Jean Edouard Baker
Message: no such element: Unable to locate element: {"method":"css selector","selector":"span.table-birthday"}
  (Session info: chrome=79.0.3945.88)

['Benoit', 'Paire']  -->  ['B']
Paire B.: Benoit Paire
['Dino', 'Marcan']  -->  ['D']
Marcan D.: Dino Marcan
['Augusto', 'Elias']  -->  ['G']
['David', 'De', 'Leon', 'Elias']  -->  ['G']
['Jose', 'Elias', 'Yapur']  -->  ['G']
['Erik', 'Eliasson']  -->  ['G']
['Erik', 'Eliasson']  -->  ['G']
['Esteban', 'Elias']  -->  ['G']
['Fredrik', 'Eliasson']  -->  ['G']
['Gastao', 'Elias']  -->  ['G']
Elias G.: Gastao Elias
['A.', 'Reid']  -->  ['M']
['A.', 'Reid']  -->  ['M']
['D.', 'Reid']  -->  ['M']
['Delroy', 'Reid']  -->  ['M']
['Doug', 'Reid']  -->  ['M']
['F.', 'Reid'

Message: no such element: Unable to locate element: {"method":"css selector","selector":"span.table-birthday"}
  (Session info: chrome=79.0.3945.88)

['Botic', 'Van', 'de', 'Zandschulp']  -->  ['D', 'V', 'B']
Zandschulp D.V.B. not found
['Albert', 'Lopez', 'Morales']  -->  ['G', 'G']
['Alberto', 'Lopez', 'Nunez']  -->  ['G', 'G']
['Daniel', 'Alejandro', 'Lopez', 'Cassaccia']  -->  ['G', 'G']
['Alex', 'Lopez', 'Moron']  -->  ['G', 'G']
['Alvaro', 'Lopez', 'San', 'Martin']  -->  ['G', 'G']
['Andres', 'Ariel', 'Fernandez', 'Lopez']  -->  ['G', 'G']
['Andres', 'Lopez']  -->  ['G', 'G']
['Javier', 'Andres', 'Lopez', 'Martinez']  -->  ['G', 'G']
['Andres', 'Munoz', 'Lopez']  -->  ['G', 'G']
['Angel', 'Luis', 'Lopez', 'Guillen']  -->  ['G', 'G']
['Miguel', 'Angel', 'Lopez', 'Jaen']  -->  ['G', 'G']
['Miguel', 'Angel', 'Lopez']  -->  ['G', 'G']
['Angel', 'Luis', 'Lopez']  -->  ['G', 'G']
['Marco', 'Antonio', 'Lopez']  -->  ['G', 'G']
['Aramis', 'Lopez']  -->  ['G', 'G']
['Benjamin', 'Winter', 

['A.', 'Smith']  -->  ['P', 'J']
['Alfonso', 'Smith']  -->  ['P', 'J']
['Andrew', 'Smith']  -->  ['P', 'J']
['Arthur', 'Smith']  -->  ['P', 'J']
['Austin', 'Smith']  -->  ['P', 'J']
['B.', 'Smith']  -->  ['P', 'J']
['Barnaby', 'Smith']  -->  ['P', 'J']
['Benjamin', 'Smith']  -->  ['P', 'J']
['Brandon', 'Weir', 'Smith']  -->  ['P', 'J']
['C.', 'Smith', 'Bingham']  -->  ['P', 'J']
['C.', 'Smith', 'Bingham']  -->  ['P', 'J']
['C.', 'Smith']  -->  ['P', 'J']
['Cameron', 'Edward', 'Smith']  -->  ['P', 'J']
['Casey', 'Smith']  -->  ['P', 'J']
['Cole', 'Smith']  -->  ['P', 'J']
['Colin', 'Smith']  -->  ['P', 'J']
['Colter', 'Smith']  -->  ['P', 'J']
['Connor', 'Smith']  -->  ['P', 'J']
['D.', 'Smith']  -->  ['P', 'J']
['Daniel', 'Smith']  -->  ['P', 'J']
['Darrell', 'Smith']  -->  ['P', 'J']
['David', 'Smith']  -->  ['P', 'J']
['Doug', 'Smith']  -->  ['P', 'J']
['Drew', 'Smith']  -->  ['P', 'J']
['Dudley', 'Smith']  -->  ['P', 'J']
['E.', 'Smith']  -->  ['P', 'J']
['Errol', 'Smith']  -->  ['P

['Miguel', 'Angel', 'Traverso']  -->  ['G', 'D']
['Victor', 'Gimeno', 'Traver']  -->  ['G', 'D']
Traver G.D. not found
['Marc', 'Andrea', 'Huesler']  -->  ['A', 'M']
Huesler A.M. not found
['Manuel', 'Guinard']  -->  ['M']
Guinard M.: Manuel Guinard
['Konstantin', 'Kotov']  -->  ['P']
['Pavel', 'Kotov']  -->  ['P']
Kotov P.: Pavel Kotov
['Javier', 'Barranco', 'Cosano']  -->  ['B', 'J']
Cosano B.J. not found
['Uladzimir', 'Ignatik']  -->  ['U']
Ignatik U.: Uladzimir Ignatik
['Martin', 'Redlicki']  -->  ['M']
Redlicki M.: Martin Redlicki
['Antoine', 'Escoffier']  -->  ['A']
Escoffier A.: Antoine Escoffier
['Arthur', 'Heller']  -->  ['P']
['Peter', 'Heller']  -->  ['P']
Heller P.: Peter Heller
['Jordi', 'Blanchar']  -->  ['U']
['Ugo', 'Blanchet']  -->  ['U']
Blanch U.: Ugo Blanchet
string index out of range
['Chia', 'Cheng', 'Yang']  -->  ['H', 'T']
['Shao', 'Chi', 'Yang']  -->  ['H', 'T']
['Dae', 'Yang', 'Kweon']  -->  ['H', 'T']
['Zi', 'Dong', 'Yang']  -->  ['H', 'T']
['Eui', 'Yeol', 'Y

['Gordan', 'Brkic']  -->  ['T']
['Josip', 'Brkic']  -->  ['T']
['Tomislav', 'Brkic']  -->  ['T']
Brkic T.: Tomislav Brkic
['Isao', 'Uchida']  -->  ['K']
['Kaichi', 'Uchida']  -->  ['K']
Uchida K.: Kaichi Uchida
['Bang', 'Ho', 'Song']  -->  ['E']
['Dong', 'Wook', 'Song']  -->  ['E']
['Evan', 'Song']  -->  ['E']
Song E.: Evan Song
['Edward', 'Bourchier']  -->  ['H']
['Harry', 'Bourchier']  -->  ['H']
Bourchier H.: Harry Bourchier
['Matias', 'Franco', 'Descotte']  -->  ['F', 'M']
Descotte F.M. not found
['Brandon', 'Blasco', 'del', 'Cid']  -->  ['R']
['Roberto', 'Cid', 'Subervi']  -->  ['R']
Cid R.: Roberto Cid Subervi
['Maverick', 'Banes']  -->  ['M']
Banes M.: Maverick Banes
['Renta', 'Tokuda']  -->  ['R']
Tokuda R.: Renta Tokuda
['Alexander', 'Zhurbin']  -->  ['A']
Zhurbin A.: Alexander Zhurbin
['Fabien', 'Reboul']  -->  ['F']
Reboul F.: Fabien Reboul
['Jacob', 'Grills']  -->  ['J']
Grills J.: Jacob Grills
Message: stale element reference: element is not attached to the page document
 

In [10]:
df_ext = pd.read_excel('data/base_data.xlsx')
scraped_data = pd.read_csv('data/playersdata.csv', encoding='utf-8-sig')

In [11]:
def fill_values(row, scraped): 
    wElo = None
    wBirthDate = None
    wHand = None
    wBackhand = None
    wSurfaceElo = None

    lElo = None
    lBirthDate = None
    lHand = None
    lBackhand = None
    lSurfaceElo = None
        
    wFound = False
    lFound = False
    for i, r in scraped.iterrows() :
        if(row['Winner']  == r['Player']) :
            wFound = True
            wElo = r['Elo']
            wBirthDate = r['BirthDate']
            wHand = r['Hand']
            wBackhand = r['BackHand']
            if(row['Surface'] == 'Hard') :
                wSurfaceElo = r['hElo']
            if(row['Surface'] == 'Clay') :
                wSurfaceElo = r['cElo']
            if(row['Surface'] == 'Grass') :
                wSurfaceElo = r['gElo']
            
        elif (row['Loser']  == r['Player']) :
            lFound = True
            lElo = r['Elo']
            lBirthDate = r['BirthDate']
            lHand = r['Hand']
            lBackhand = r['BackHand']
            if(row['Surface'] == 'Hard') :
                lSurfaceElo = r['hElo']
            if(row['Surface'] == 'Clay') :
                lSurfaceElo = r['cElo']
            if(row['Surface'] == 'Grass') :
                lSurfaceElo = r['gElo']
        elif wFound and lFound :
            break
        
    print(row.name, row['Winner'], row['Loser'])
    return wElo, wSurfaceElo, wBirthDate, wHand, wBackhand, lElo, lSurfaceElo, lBirthDate, lHand, lBackhand

In [12]:
def join_datasets(base, scraped):
    result = base.apply(lambda row: fill_values(row, scraped), axis=1, result_type='expand')
            
    base = base.assign(WElo=result[0], WSurfElo=result[1], WBD=result[2], WHand=result[3], WBHand=result[4],
                  LElo=result[5], LSurfElo=result[6], LBD=result[7], LHand=result[8], LBHand=result[9])
    
    base['Date'] = pd.to_datetime(base['Date'])
    return base.sort_values(by='Date')

def write_dataset(base):
    os.chdir('data')
    with open('dataset.csv', 'w', newline='', encoding='utf-8-sig') as file :
        writer = csv.writer(file)
        writer.writerow([c for c in base.columns])
        for i, r in df_ext.iterrows():
            writer.writerow(r)
    os.chdir('..')
            
def split_by_year(base):
    os.chdir('data')
    for year in base.Date.dt.year.unique():
        base[base.Date.dt.year == year].to_csv(str(year) + '.csv', 
                                               encoding='utf-8-sig',
                                               index=False)
    os.chdir('..')

In [13]:
df_ext = join_datasets(df_ext, scraped_data)
write_dataset(df_ext)
split_by_year(df_ext)

0 Dimitrov G. Nishioka Y.
1 Raonic M. Bedene A.
2 Kecmanovic M. Mayer L.
3 Millman J. Sandgren T.
4 Uchiyama Y. Humbert U.
5 Kudla D. Fritz T.
6 Chardy J. Struff J.L.
7 Murray A. Duckworth J.
8 Kyrgios N. Harrison R.
9 Tsonga J.W. Kokkinakis T.
10 De Minaur A. Popyrin A.
11 Thompson J. Bolt A.
12 Dimitrov G. Millman J.
13 Uchiyama Y. Edmund K.
14 Nishikori K. Kudla D.
15 Raonic M. Kecmanovic M.
16 Chardy J. Kyrgios N.
17 Medvedev D. Murray A.
18 De Minaur A. Thompson J.
19 Tsonga J.W. Daniel T.
20 Chardy J. Uchiyama Y.
21 Nishikori K. Dimitrov G.
22 Medvedev D. Raonic M.
23 Tsonga J.W. De Minaur A.
24 Nishikori K. Chardy J.
25 Medvedev D. Tsonga J.W.
26 Nishikori K. Medvedev D.
27 Dolgopolov O. Schwartzman D.
28 De Minaur A. Johnson S.
29 Harrison R. Mayer L.
30 Ebden M. Tiafoe F.
31 Zverev M. Smith J.P.
32 Zeballos H. Escobedo E.
33 Donaldson J. Thompson J.
34 Mmoh M. Delbonis F.
35 Chung H. Muller G.
36 Edmund K. Shapovalov D.
37 Istomin D. Dzumhur D.
38 Millman J. Polansky P.
39 Dol

316 Kovalik J. Elias G.
317 Chung H. Coric B.
318 Olivo R. Ruud C.
319 Bedene A. Garcia-Lopez G.
320 Lu Y.H. Albot R.
321 Paire B. Kravchuk K.
322 Bhambri Y. Ramanathan R.
323 Dutra Silva R. Lajovic D.
324 Youzhny M. Myneni S.
325 Medvedev D. Lu Y.H.
326 Ramos-Vinolas A. Darcis S.
327 Kovalik J. Cilic M.
328 Sela D. Chung H.
329 Youzhny M. Olivo R.
330 Bautista Agut R. Dutra Silva R.
331 Paire B. Bhambri Y.
332 Bedene A. Klizan M.
333 Medvedev D. Kovalik J.
334 Sela D. Ramos-Vinolas A.
335 Paire B. Bedene A.
336 Bautista Agut R. Youzhny M.
337 Medvedev D. Sela D.
338 Bautista Agut R. Paire B.
339 Bautista Agut R. Medvedev D.
340 Millman J. Donskoy E.
341 Pavic A. Almagro N.
342 Coric B. Granollers M.
343 Garcia-Lopez G. Daniel T.
344 Rublev A. Devvarman S.
345 Krajicek A. Ram R.
346 Rosol L. Giraldo S.
347 Ramanathan R. Gimeno-Traver D.
348 Vanni L. Struff J.L.
349 Muller G. Khachanov K.
350 Fabbiano T. Kovalik J.
351 Bedene A. Pospisil V.
352 Coric B. Krajicek A.
353 Garcia-Lopez G. P

626 Verdasco F. Gabashvili T.
627 Karlovic I. Rosol L.
628 Berdych T. Istomin D.
629 Seppi A. Mayer L.
630 Kavcic B. Monaco J.
631 Stakhovsky S. Ali Mutawa J.M.
632 Basilashvili N. Youzhny M.
633 Djokovic N. Lajovic D.
634 Struff J.L. Kohlschreiber P.
635 Berrer M. Nadal R.
636 Souza J. Jaziri M.
637 Ferrer D. De Bakker T.
638 Karlovic I. Basilashvili N.
639 Brown D. Struff J.L.
640 Seppi A. Souza J.
641 Ferrer D. Verdasco F.
642 Djokovic N. Stakhovsky S.
643 Dodig I. Berrer M.
644 Berdych T. Kavcic B.
645 Gasquet R. Bolelli S.
646 Ferrer D. Brown D.
647 Karlovic I. Djokovic N.
648 Berdych T. Gasquet R.
649 Seppi A. Dodig I.
650 Ferrer D. Karlovic I.
651 Berdych T. Seppi A.
652 Ferrer D. Berdych T.
653 Monfils G. Giraldo S.
654 Kubot L. Gimeno-Traver D.
655 Karlovic I. Berdych T.
656 Verdasco F. Volandri F.
657 Hanescu V. Sousa J.
658 Kamke T. Jaziri M.
659 Brown D. Dodig I.
660 Gojowczyk P. Thiem D.
661 Murray A. Zayed M.S.
662 Brands D. Davydenko N.
663 Ferrer D. Dolgopolov O.
664 Ma

938 Isner J. Lu Y.H.
939 Bellucci T. Goffin D.
940 Malisse X. Klizan M.
941 Rochus O. Ramos A.
942 Baker B. Janowicz J.
943 Sijsling I. Haase R.
944 Lacko L. Lorenzi P.
945 Giraldo S. Soeda G.
946 Jones G. Melzer J.
947 Falla A. Zemlja G.
948 Levine J. King-Turner D.
949 Monfils G. Becker B.
950 Lu Y.H. Paire B.
951 Querrey S. Rochus O.
952 Malisse X. Giraldo S.
953 Kohlschreiber P. Falla A.
954 Lacko L. Bellucci T.
955 Levine J. Baker B.
956 Monfils G. Jones G.
957 Haas T. Sijsling I.
958 Ferrer D. Lu Y.H.
959 Kohlschreiber P. Malisse X.
960 Querrey S. Levine J.
961 Ferrer D. Lacko L.
962 Monfils G. Haas T.
963 Kohlschreiber P. Querrey S.
964 Ferrer D. Monfils G.
965 Ferrer D. Kohlschreiber P.
966 Falla A. Young D.
967 Bellucci T. Machado R.
968 Giraldo S. Venus M.
969 Berlocq C. Volandri F.
970 Garcia-Lopez G. Kunitsyn I.
971 Mannarino A. Anderson K.
972 Kohlschreiber P. Bohli S.
973 Rochus O. Kamke T.
974 Rosol L. Ramos A.
975 Harrison R. Querrey S.
976 Starace P. Montanes A.
977 Pa

1239 Andreev I. Chardy J.
1240 Gil F. Nieminen J.
1241 Chela J.I. Stepanek R.
1242 Garcia-Lopez G. Guccione C.
1243 Starace P. Benneteau J.
1244 Seppi A. Istomin D.
1245 Gasquet R. Ebden M.
1246 Simon G. Lu Y.H.
1247 Dolgopolov O. Tomic B.
1248 Troicki V. Chela J.I.
1249 Gulbis E. Andreev I.
1250 Stakhovsky S. Garcia-Lopez G.
1251 Starace P. Kubot L.
1252 Simon G. Gil F.
1253 Gasquet R. Seppi A.
1254 Mayer F. Del Potro J.M.
1255 Dolgopolov O. Querrey S.
1256 Simon G. Dolgopolov O.
1257 Gulbis E. Stakhovsky S.
1258 Troicki V. Gasquet R.
1259 Mayer F. Starace P.
1260 Simon G. Gulbis E.
1261 Troicki V. Mayer F.
1262 Simon G. Troicki V.
1263 Anderson K. Mannarino A.
1264 Basilashvili N. Eubanks C.
1265 Albot R. Mmoh M.
1266 Travaglia S. Andreozzi G.
1267 Verdasco F. Kecmanovic M.
1268 Cuevas P. Lajovic D.
1269 Haase R. Garcia-Lopez G.
1270 Tsitsipas S. Berrettini M.
1271 Dimitrov G. Tipsarevic J.
1272 Tiafoe F. Gunneswaran P.
1273 Nadal R. Duckworth J.
1274 Fabbiano T. Kubler J.
1275 Opelk

...

22426 Mannarino A. Seppi A.
22427 Khachanov K. Medvedev D.
22428 Khachanov K. Mannarino A.
22429 Sela D. Coric B.
22430 Bhambri Y. Safiullin R.
22431 Bublik A. Gabashvili T.
22432 Dzumhur D. Fabbiano T.
22433 Rosol L. Kravchuk K.
22434 Dutra Silva R. Donskoy E.
22435 Seppi A. Vesely J.
22436 Djere L. Lorenzi P.
22437 Berankis R. Lajovic D.
22438 Krajinovic F. Rublev A.
22439 Basic M. Kavcic B.
22440 Medvedev D. Kuznetsov An.
22441 Sela D. Dutra Silva R.
22442 Dzumhur D. Bhambri Y.
22443 Mannarino A. Rosol L.
22444 Seppi A. Kohlschreiber P.
22445 Basic M. Djere L.
22446 Medvedev D. Carreno Busta P.
22447 Berankis R. Krajinovic F.
22448 Bublik A. Ramos-Vinolas A.
22449 Dzumhur D. Seppi A.
22450 Mannarino A. Sela D.
22451 Basic M. Medvedev D.
22452 Berankis R. Bublik A.
22453 Dzumhur D. Basic M.
22454 Berankis R. Mannarino A.
22455 Dzumhur D. Berankis R.
22456 Harrison R. Pavlasek A.
22457 Mannarino A. Young D.
22458 Albot R. Johnson S.
22459 Anderson K. Baghdatis M.
22460 Almagro N. Sugi

22716 Fognini F. Sock J.
22717 Dimitrov G. Zverev M.
22718 Del Potro J.M. Sugita Y.
22719 Verdasco F. Anderson K.
22720 Dimitrov G. Fognini F.
22721 Del Potro J.M. Verdasco F.
22722 Del Potro J.M. Dimitrov G.
22723 Delbonis F. Kuznetsov An.
22724 Nishikori K. Lajovic D.
22725 Young D. Marchenko I.
22726 Cilic M. Youzhny M.
22727 Granollers M. Laaksonen H.
22728 Muller G. Dimitrov G.
22729 Lorenzi P. Mahut N.
22730 Berankis R. Raonic M.
22731 Wawrinka S. Chiudinelli M.
22732 Mayer F. Paire B.
22733 Goffin D. Baghdatis M.
22734 Sock J. Mannarino A.
22735 Carreno Busta P. Verdasco F.
22736 Pella G. Gasquet R.
22737 Zverev M. Fritz T.
22738 Del Potro J.M. Haase R.
22739 Nishikori K. Lorenzi P.
22740 Delbonis F. Berankis R.
22741 Muller G. Mayer F.
22742 Granollers M. Sock J.
22743 Cilic M. Carreno Busta P.
22744 Del Potro J.M. Goffin D.
22745 Wawrinka S. Young D.
22746 Zverev M. Pella G.
22747 Muller G. Delbonis F.
22748 Nishikori K. Del Potro J.M.
22749 Zverev M. Wawrinka S.
22750 Cilic M

23004 Sock J. Haase R.
23005 Goffin D. Chung H.
23006 Fucsovics M. Mayer L.
23007 Del Potro J.M. Benneteau J.
23008 Cilic M. Coric B.
23009 Federer R. Paire B.
23010 Mannarino A. Shapovalov D.
23011 Del Potro J.M. Bautista Agut R.
23012 Cilic M. Fucsovics M.
23013 Federer R. Mannarino A.
23014 Goffin D. Sock J.
23015 Del Potro J.M. Cilic M.
23016 Federer R. Goffin D.
23017 Federer R. Del Potro J.M.
23018 Khachanov K. Seppi A.
23019 Karlovic I. Johnson S.
23020 Dzumhur D. Almagro N.
23021 Isner J. Struff J.L.
23022 Basilashvili N. Berdych T.
23023 Ferrer D. Edmund K.
23024 Troicki V. Anderson K.
23025 Lopez F. Pouille L.
23026 Simon G. Garcia-Lopez G.
23027 Tsonga J.W. Becker B.
23028 Sousa J. Cuevas P.
23029 Thiem D. Melzer G.
23030 Melzer J. Bautista Agut R.
23031 Kohlschreiber P. Robert S.
23032 Ramos-Vinolas A. Fognini F.
23033 Murray A. Klizan M.
23034 Isner J. Lopez F.
23035 Karlovic I. Dzumhur D.
23036 Khachanov K. Basilashvili N.
23037 Tsonga J.W. Kohlschreiber P.
23038 Ramos-Vi

23293 Khachanov K. Novak D.
23294 Kukushkin M. Dimitrov G.
23295 Fucsovics M. Fognini F.
23296 Anderson K. Melzer J.
23297 Coric B. Pouille L.
23298 Nishikori K. Khachanov K.
23299 Verdasco F. Edmund K.
23300 Kukushkin M. Rublev A.
23301 Thiem D. Querrey S.
23302 Monfils G. Isner J.
23303 Kukushkin M. Fucsovics M.
23304 Nishikori K. Thiem D.
23305 Anderson K. Coric B.
23306 Verdasco F. Monfils G.
23307 Nishikori K. Kukushkin M.
23308 Anderson K. Verdasco F.
23309 Anderson K. Nishikori K.
23310 Simon G. Gulbis E.
23311 Gasquet R. Lopez F.
23312 Herbert P.H. Tomic B.
23313 Kohlschreiber P. Isner J.
23314 Zverev A. Troicki V.
23315 Edmund K. Ferrer D.
23316 Novak D. Fabbiano T.
23317 Garcia-Lopez G. Anderson K.
23318 Ramos-Vinolas A. Querrey S.
23319 Pouille L. Ofner S.
23320 Thiem D. Rublev A.
23321 Struff J.L. Cuevas P.
23322 Carreno Busta P. Pella G.
23323 Schwartzman D. Fognini F.
23324 Dzumhur D. Berankis R.
23325 Tsonga J.W. Khachanov K.
23326 Kohlschreiber P. Herbert P.H.
23327 Zve

23583 Raonic M. Thiem D.
23584 Murray A. Wawrinka S.
23585 Cilic M. Nishikori K.
23586 Murray A. Raonic M.
23587 Djokovic N. Nishikori K.
23588 Murray A. Djokovic N.
23589 Djokovic N. Nishikori K.
23590 Federer R. Berdych T.
23591 Murray A. Ferrer D.
23592 Nadal R. Wawrinka S.
23593 Nishikori K. Berdych T.
23594 Federer R. Djokovic N.
23595 Nadal R. Murray A.
23596 Wawrinka S. Ferrer D.
23597 Federer R. Nishikori K.
23598 Djokovic N. Berdych T.
23599 Nadal R. Ferrer D.
23600 Wawrinka S. Murray A.
23601 Djokovic N. Nadal R.
23602 Federer R. Wawrinka S.
23603 Djokovic N. Federer R.
23604 Anderson K. Thiem D.
23605 Nishikori K. Federer R.
23606 Zverev A. Cilic M.
23607 Djokovic N. Isner J.
23608 Anderson K. Nishikori K.
23609 Federer R. Thiem D.
23610 Djokovic N. Zverev A.
23611 Cilic M. Isner J.
23612 Thiem D. Nishikori K.
23613 Federer R. Anderson K.
23614 Zverev A. Isner J.
23615 Djokovic N. Cilic M.
23616 Zverev A. Federer R.
23617 Djokovic N. Anderson K.
23618 Zverev A. Djokovic N.
2